In [ ]:
!pip install qiskit qiskit-aer statsmodels scipy matplotlib

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit.providers.aer import AerSimulator
from qiskit.providers.aer.noise import NoiseModel, depolarizing_error

import numpy as np
from statsmodels.stats.proportion import proportion_confint
from scipy.stats import binom_test

import matplotlib.pyplot as plt

In [ ]:
# Clifford-only circuit
circ_cliff = QuantumCircuit(3, 3)
circ_cliff.h(0)
circ_cliff.cx(0, 1)
circ_cliff.cx(1, 2)
circ_cliff.barrier()
circ_cliff.measure([0, 1, 2], [0, 1, 2])  # Ideal '000'

# Non-Clifford circuit
circ_noncliff = QuantumCircuit(3, 3)
circ_noncliff.h(0)
circ_noncliff.t(1)
circ_noncliff.cx(1, 2)
circ_noncliff.barrier()
circ_noncliff.measure([0, 1, 2], [0, 1, 2])  # Ideal '110'

In [ ]:
# Create depolarizing noise model
noise_model = NoiseModel()
error_1q = depolarizing_error(0.01, 1)
error_2q = depolarizing_error(0.02, 2)
noise_model.add_all_qubit_quantum_error(error_1q, ['u1','u2','u3','x','h'])
noise_model.add_all_qubit_quantum_error(error_2q, ['cx'])

# Initialize Aer simulator
sim = AerSimulator(noise_model=noise_model)

In [ ]:
threshold = 0.90
shots_per_iter = 50
max_iters = 20

results = {}

for name, circ in [('Clifford', circ_cliff), ('Non-Clifford', circ_noncliff)]:
    total_success = 0
    lower_bounds = []
    x_vals = []

    for i in range(max_iters):
        result = sim.run(transpile(circ, sim), shots=shots_per_iter).result()
        counts = result.get_counts()
        target = '000' if name == 'Clifford' else '110'
        succ = counts.get(target, 0)
        total_success += succ

        total_shots = shots_per_iter * (i + 1)
        lb, ub = proportion_confint(total_success, total_shots, alpha=0.05, method='wilson')
        lower_bounds.append(lb)
        x_vals.append(total_shots)

        p_val = binom_test(total_success, total_shots, p=threshold, alternative='greater')
        if p_val > 0.95:
            break

    results[name] = (x_vals, lower_bounds)

In [ ]:
plt.figure(figsize=(7,4))
for name, (x, lb) in results.items():
    plt.plot(x, lb, marker='o', label=f'{name} Circuit')

plt.axhline(y=threshold, color='r', linestyle='--', label='Threshold = 0.90')
plt.xlabel('Total Shots')
plt.ylabel('Lower 95% Confidence Bound')
plt.title('Adaptive Verification Confidence vs Samples')
plt.legend()
plt.grid(True)
plt.show()